## This script should demonstrate the main functions

In [1]:
# from datetime import datetime
# from sklearn.metrics import accuracy_score, f1_score, recall_score
import os
from openai import OpenAI
from dotenv import load_dotenv
import random
import json
from datetime import datetime, timedelta
from utils.evaluation_utils import evaluate_label_single, calculate_overall_metrics
from utils.generation_utils import generate_prompt, label2email

In [2]:
pred_path = "./data/test_examples/test_pred.json"
true_path = "./data/test_examples/test_true.json"
with open(pred_path, 'r') as file:
    pred_labels = json.load(file)
with open(true_path, 'r') as file:
    true_labels = json.load(file)

In [3]:
pred_labels

[{'Spam': 'No',
  'Subject': 'Random Subject 85',
  'Sender': 'sender46@example.com',
  'send_date': '2024-03-12 14:45',
  'Time_Sensitive': 'Yes',
  'Start': '2024-11-20 09:17',
  'End': '2024-11-20 11:20',
  'Type': 'Event',
  'Category': 'Work',
  'Format': 'Online',
  'Location': 'Location 7',
  'Action_Required': 'Yes',
  'Priority_Level': 'High'},
 {'Spam': 'Yes',
  'Subject': 'Random Subject 29',
  'Sender': 'sender31@example.com',
  'send_date': '2024-09-09 03:19',
  'Time_Sensitive': 'No',
  'Start': '2024-07-18 10:33',
  'End': '2024-07-18 11:33',
  'Type': 'Reminder',
  'Category': 'Study',
  'Format': 'In-person',
  'Location': 'Location 5',
  'Action_Required': 'No',
  'Priority_Level': 'Medium'},
 {'Spam': 'No',
  'Subject': 'Random Subject 10',
  'Sender': 'sender89@example.com',
  'send_date': '2024-01-21 16:21',
  'Time_Sensitive': 'Yes',
  'Start': '2024-05-03 09:45',
  'End': '2024-05-03 10:45',
  'Type': 'Event',
  'Category': 'Leisure',
  'Format': 'Online',
  'Loc

In [4]:
results = []
assert len(pred_labels) == len(true_labels)
for i in range(len(pred_labels)):
    results.append(evaluate_label_single(pred_labels[i], true_labels[i]))
metrics = calculate_overall_metrics(results)
metrics

{'Binary Metrics': {'Spam': {'ACC': 0.8,
   'F1': 0.8888888888888888,
   'Recall': 0.8},
  'Time_Sensitive': {'ACC': 1.0, 'F1': 1.0, 'Recall': 1.0}},
 'Categorical Metrics': {'Time Period': 0.8,
  'Type': 1.0,
  'Category': 1.0,
  'Format': 1.0,
  'Priority_Level': 1.0},
 'Averaged Weighted Score': 0.76}

In [5]:
# Load the environment variables from the .env file
# In this .env, it contains openai's API Key.
load_dotenv()
# Load the key to call the client.
client = OpenAI()
model_name="gpt-4o-mini"


In [6]:
sys_prompt, task_prompt = generate_prompt(true_labels[0])
label2email(label=true_labels[0], client=client, temperature=1.0, model=model_name)

'From: sender46@example.com  \nDate: 2024-11-20  \nSubject: Random Subject 85  \n\nDear Team,  \n\nI hope this message finds you well. I wanted to inform you about an upcoming online session scheduled for November 20, 2024, from 9:17 AM to 11:25 AM. We will be meeting virtually at Location 7.  \n\nPlease confirm your attendance at your earliest convenience, and do not hesitate to reach out if you have any questions or need assistance accessing the session. Your participation is highly valued, and I look forward to seeing everyone online.  \n\nThank you for your cooperation!  \n\nBest,  \n[Your Name]  \nsender46@example.com  '